# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
# sys
import os
import glob

# text and data
import re
import cassandra
import numpy as np
import pandas as pd

# file formats
#import json
import csv

## Pandas factory from https://stackoverflow.com/questions/41247345/python-read-cassandra-data-into-pandas
## for using Panda dataframes to add columns to the query output
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking the current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))
# see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra Setup

#### Creating a Cluster

In [5]:
# Connection to a Cassandra instance
# (127.0.0.1)
try:
    from cassandra.cluster import Cluster
    cluster = Cluster(['127.0.0.1'])
    
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    
except Exception as e:
        print(e)

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
keyspace_query = """
                    CREATE KEYSPACE IF NOT EXISTS udacity_proj2 
                    WITH REPLICATION = {'class': 'SimpleStrategy'
                                      , 'replication_factor': 1 }
                 """
try:
    session.execute(keyspace_query)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity_proj2')
except Exception as e:
    print(e)

# Part III. Table Creation and Primary Key Logic

### 1 - Create the table for artist, song title, song length with sessionId and itemInSession features. 
The purpose is to gather artist, song title and length data based on the sessionId = 338, and itemInSession = 4 conditions.

In [20]:
query_sessionAndItem_table = """
                CREATE TABLE IF NOT EXISTS session_and_item (
                    sessionId INT
                  , itemInSession INT
                  , artist_name TEXT
                  , song_title TEXT
                  , song_length FLOAT
                  , PRIMARY KEY (sessionId, itemInSession))
               """

try:
    session.execute(query_sessionAndItem_table)
except Exception as e:
    print(e)

#### Primary Key Selection Logic:
I pick sessionID and itemInSession for the primary key because they match the requested query pattern (the conditions). To dig in a little bit more, I decided to use `sessionId` instead of `itemInSession` as the Partition Key because it offers a better data distribution between the nodes than `itemInSession`. `sessionID`'s levels of uniqueness and randomness seems higher since the intergers are larger than the `itemInSession`'s which can indicate less repetitiveness and more randomness based on the size of the dataset. Once we focus on a certain session ID, the Clustering Key, `itemInSession` seems like a better fit to assit with navigation of the table.

### 2 - Create the table for artist, song title (sorted by `itemInSession`) and user. 
The purpose is to gather artist, song title and length data based on the userId = 10, and SessionId = 182 conditions.

In [21]:
query_sessionAndUser_table = """
                CREATE TABLE IF NOT EXISTS session_and_user (
                    userId INT
                  , sessionId INT
                  , itemInSession INT
                  , artist_name TEXT
                  , song_title TEXT
                  , user_first TEXT
                  , user_last TEXT
                  , PRIMARY KEY ((userId, sessionId), itemInSession))
               """

try:
    session.execute(query_sessionAndUser_table)
except Exception as e:
    print(e)              

#### Primary Key Selection Logic:
The requested query highlights them for me. I include the two features from the conditions indicated (`userId` and `sessionId`). And, I added `itemInSession` since it was needed to add another level of uniqueness since one user can be in the same session but engagin different items in that same session. Initially, I used `userId` as the Partition Key by itself along with `sessionId` and `itemInSession` as Clustering Keys. However, I was not able to sort the results due to the Error caused by unmatched order of columns to Primary Key. Actually, having `sessionId` as one of the Clustering Keys do not make sense based on the selection I perfomed for the previous query (randomness). So, I included `sessionId` in the Partition Key. 

### 3 - Create the table for user. 
The purpose is to capture all the users who listened song_title = 'All Hands Against His Own'.

In [22]:
query_userAndSong_table = """
                CREATE TABLE IF NOT EXISTS user_and_song (
                    song_title TEXT
                  , userId INT
                  , user_first TEXT
                  , user_last TEXT
                  , PRIMARY KEY (song_title, userId, user_last, user_first))
               """

try:
    session.execute(query_userAndSong_table)
except Exception as e:
    print(e)              

 #### Primary Key Selection
I thought the best way to partition this version would be by `song_title` since the request was to select user listening a particular song (targeting a certain partition would allow us to get results faster). The Clustering Keys are easier to decide in this exercise, in this case it is user's first and last name. I put first name before last name given that it may add a bit more speed to query since first name will show less randomness than last name. 

# Part IV. Loading

In [28]:
# initilize the file
file = 'event_datafile_new.csv'

# queries
query_sessionAndItem_insert = """INSERT INTO session_and_item (sessionId, itemInSession, artist_name, song_title, song_length) 
                                 VALUES (%s, %s, %s, %s, %s)"""
query_sessionAndUser_insert = """INSERT INTO session_and_user (userId, sessionId, itemInSession, artist_name, song_title, user_first, user_last)
                                 VALUES (%s, %s, %s, %s, %s, %s, %s)"""
query_userAndSong_insert = """INSERT INTO user_and_song (song_title, userId, user_first, user_last)
                              VALUES (%s, %s, %s, %s)"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # insertion
        session.execute(query_sessionAndItem_insert, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        session.execute(query_sessionAndUser_insert, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        session.execute(query_userAndSong_insert, (line[9], int(line[10]), line[1], line[4]))

# Part V. Verification

Let's verify that the data has been inserted in the tables. First inilize the Pandas and set the queries.

In [29]:
# use Pandas to print result
session.row_factory = pandas_factory

# queries
query_sessionAndItem_result = "SELECT artist_name, song_title, song_length \
                               FROM session_and_item \
                               WHERE sessionId = 338 \
                                   AND itemInSession = 4"
query_sessionAndUser_result = "SELECT artist_name, song_title, user_first, user_last \
                               FROM session_and_user \
                               WHERE userId = 10 AND SessionId = 182 \
                               ORDER BY itemInSession"
query_userAndSong_result = "SELECT user_first, user_last \
                            FROM user_and_song \
                            WHERE song_title = 'All Hands Against His Own'"

**First query:** The artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [30]:
result = session.execute(query_sessionAndItem_result)
df = result._current_rows
df

,artist_name,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


**Second query:** The name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [31]:
result = session.execute(query_sessionAndUser_result)
df = result._current_rows
df

,artist_name,song_title,user_first,user_last
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


**Third query:** Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [32]:
result = session.execute(query_userAndSong_result)
df = result._current_rows
df

,user_first,user_last
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


# Part VI. Drop Tables And Close Session & Cluster Connection

Drop the tables before closing out the sessions.

In [33]:
table_list = ['user_and_song', 'session_and_user', 'session_and_item']

for e in table_list:
    query_drop = f"DROP TABLE IF EXISTS {e}"
    try:
        rows = session.execute(query_drop)
    except Exception as e:
        print(e)

Close the session and cluster connection.

In [34]:
session.shutdown()
cluster.shutdown()

# THE END!